In [1]:
!pip install tensorflow-datasets --quiet
!pip install pydot --quiet
!pip install transformers --quiet

     |████████████████████████████████| 5.5 MB 4.5 MB/s 
     |████████████████████████████████| 7.6 MB 42.7 MB/s 
     |████████████████████████████████| 182 kB 76.1 MB/s 


In [2]:
import numpy as np
import pandas as pd
import seaborn as sns

import tensorflow as tf
from tensorflow import keras

from tensorflow.keras.layers import Embedding, Input, Dense, Lambda
from tensorflow.keras.models import Model
import tensorflow.keras.backend as K
import tensorflow_datasets as tfds

import sklearn as sk
import os
import nltk
from nltk.data import find

import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split

#let's make longer output readable without scrolling
from pprint import pprint
import re

In [3]:
# Load the Drive helper and mount
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# define paths in colab
dataset_path = 'drive/MyDrive/Colab Notebooks/w266_project_data'
csv_path = 'drive/MyDrive/Colab Notebooks/w266_project_predictions/'
model_path = 'drive/MyDrive/Colab Notebooks/w266_project_models/'

In [5]:
# a list of Davidson's translated model output CSV files
# column names: test_predictions,test_references
output_file_names = ['t5_large_ft.csv', 'bart_xsum_ft.csv', 'bart_cnn_ft.csv', 
                     'bart_detox_zsl.csv']

In [6]:
# a list of Davidson's classification model CSV files
class_file_names = ['davidson_bertweet_class_pred.csv', 'davidson_bert_class_pred.csv',
                    'davidson_bertlarge_class_pred.csv']

## Models

In [7]:
max_length = 25

### BERT Tweet Model

#### load BERT Tweet pretrain

In [8]:
model_checkpoint = 'vinai/bertweet-base'

In [9]:
def create_bert_model(checkpoint = model_checkpoint,
                                hidden_size = 66, 
                                dropout=0.3,
                                learning_rate=0.00005):
  """
  Build a simple classification model with BERT. Use the Pooled Output for classification purposes.
  """
  #bert_model = TFBertModel.from_pretrained(checkpoint)                                              
  ### YOUR CODE HERE

  input_ids = tf.keras.layers.Input(shape=(max_length,), dtype=tf.int64, name='input_ids_layer')
  token_type_ids = tf.keras.layers.Input(shape=(max_length,), dtype=tf.int64, name='token_type_ids_layer')
  attention_mask = tf.keras.layers.Input(shape=(max_length,), dtype=tf.int64, name='attention_mask_layer')

  bert_inputs = {'input_ids': input_ids,
  'token_type_ids': token_type_ids,
  'attention_mask': attention_mask
  }      

  bert_out = bertweet_model(bert_inputs)
  pooled_token = bert_out[1]

  hidden = tf.keras.layers.Dense(hidden_size, activation='relu', name='hidden_layer')(pooled_token)
  hidden = tf.keras.layers.Dropout(dropout, name='dropout_layer')(hidden)

  classification = tf.keras.layers.Dense(3, activation='softmax',name='classification_layer')(hidden)

  classification_model = tf.keras.Model(inputs=[input_ids, token_type_ids, attention_mask], outputs=[classification])

  classification_model.compile(
      optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
      loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False), 
      metrics=['accuracy'])


  ### END YOUR CODE
  return classification_model

In [10]:
from transformers import AutoModel, AutoTokenizer, TFAutoModel

bertweet_model = TFAutoModel.from_pretrained(model_checkpoint)
bertweet_tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=False)

Downloading:   0%|          | 0.00/558 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/740M [00:00<?, ?B/s]

Some layers from the model checkpoint at vinai/bertweet-base were not used when initializing TFRobertaModel: ['lm_head']
- This IS expected if you are initializing TFRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFRobertaModel were initialized from the model checkpoint at vinai/bertweet-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.


Downloading:   0%|          | 0.00/843k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.08M [00:00<?, ?B/s]

emoji is not installed, thus not converting emoticons or emojis into text. Install emoji: pip3 install emoji==0.6.0
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


#### load fine-tuned BERTWEET model (to compare with fine-tuned balanced BERTWEET model 

In [11]:
model_name = 'davidson_bertweet_weights.hdf5'

pooled_bertweet_model = create_bert_model()

# The model weights (that are considered the best) are loaded into the
# model.
checkpoint_file = model_path + model_name
pooled_bertweet_model.load_weights(checkpoint_file)

#### load Balanced fine-tuned BERTWEET model

In [12]:
model_name = 'davidson_bertweet_balanced_weights.hdf5'
balanced_bertweet_model = create_bert_model()

checkpoint_file = model_path + model_name
balanced_bertweet_model.load_weights(checkpoint_file)

### Load T5, XSUM, CBB and Paradetox Translated Data and run the BERTWEET classification model

In [13]:
# install huggingface datasets
!pip install datasets --quiet
from datasets import load_dataset, load_from_disk, load_metric, DatasetDict

     |████████████████████████████████| 451 kB 4.7 MB/s 
     |████████████████████████████████| 115 kB 69.8 MB/s 
     |████████████████████████████████| 212 kB 63.4 MB/s 
     |████████████████████████████████| 127 kB 70.7 MB/s 


In [14]:
# load the dataset
dataset = load_from_disk(dataset_path)
org_inputs = pd.DataFrame(dataset['test'])

In [15]:
org_inputs['en_toxic_comment']

0         holy shit , the 61 61 view must be hilarious .
1      all she has to do is smile and keep her cool w...
2      can we apply that litmus test to muslims that ...
3                          thats fucking it , im joining
4      the residents of davis have a collective stick...
                             ...                        
984    my leg still hurts like a bitch from when i hi...
985                            fuck it let 's get some !
986                  you gotta come at me sideways nshit
987    why the fuck won 't anybody do anything ? so b...
988    oh shit who told u this lie lmfao bedroombody ...
Name: en_toxic_comment, Length: 989, dtype: object

#### Classifiy org. toxic inputs

In [16]:
# run model.predict()
bertweet_encodings = bertweet_tokenizer(list(org_inputs['en_toxic_comment']), truncation=True, padding='max_length', max_length=max_length, return_tensors='tf')
un_predictions = pooled_bertweet_model.predict([bertweet_encodings.input_ids, bertweet_encodings.token_type_ids, bertweet_encodings.attention_mask])
un_prediction_classes = un_predictions.argmax(axis=-1)
balanced_predictions = balanced_bertweet_model.predict([bertweet_encodings.input_ids, bertweet_encodings.token_type_ids, bertweet_encodings.attention_mask])
balanced_prediction_classes = balanced_predictions.argmax(axis=-1)

print('unbalanced = ', np.unique(un_prediction_classes, return_counts=True))
print('balanced = ', np.unique(balanced_prediction_classes, return_counts=True))

#balanced_prediction_classes

31/31 [==============================] - 8s 77ms/step
unbalanced =  (array([0, 1, 2]), array([221, 764,   4]))
balanced =  (array([0, 1, 2]), array([548, 370,  71]))


In [17]:
org_inputs_class = balanced_prediction_classes

In [18]:
# a list of Davidson's translated model output CSV files
# column names: test_predictions,test_references
output_file_names = ['t5_large_ft.csv', 'bart_xsum_ft.csv', 'bart_cnn_ft.csv', 
                     'bart_detox_zsl.csv']
result_file_names = ['detox_bertweet_class_outoput_t5.csv','detox_bertweet_class_outoput_xsum.csv',
                     'detox_bertweet_class_outoput_cnn.csv','detox_bertweet_class_outoput_paradetox.csv']

In [19]:
for i in range(len(output_file_names)):
  #print(result_file_names[i])
  output_file_name = output_file_names[i]
  result_file_name = result_file_names[i]

  df_predictions = pd.read_csv(csv_path + output_file_name)

  # run model.predict()
  bertweet_encodings = bertweet_tokenizer(list(df_predictions['test_predictions']), truncation=True, padding='max_length', max_length=max_length, return_tensors='tf')
  un_predictions = pooled_bertweet_model.predict([bertweet_encodings.input_ids, bertweet_encodings.token_type_ids, bertweet_encodings.attention_mask])
  un_prediction_classes = un_predictions.argmax(axis=-1)
  balanced_predictions = balanced_bertweet_model.predict([bertweet_encodings.input_ids, bertweet_encodings.token_type_ids, bertweet_encodings.attention_mask])
  balanced_prediction_classes = balanced_predictions.argmax(axis=-1)

  print(output_file_name)
  print('unbalanced = ', np.unique(un_prediction_classes, return_counts=True))
  print('balanced = ', np.unique(balanced_prediction_classes, return_counts=True))

  #df_class = df_class[:2400]
  #df_predictions = df_predictions[:2400]
  #balanced_prediction_classes = balanced_prediction_classes[:2400]
  
  # save prediction output to CSV file
  save_csv_dict = {'org_input': org_inputs['en_toxic_comment'], 'pred_class': org_inputs_class, 
                   'output': df_predictions['test_predictions'], 'output_class': balanced_prediction_classes}
  df_save = pd.DataFrame(save_csv_dict) 

  # saving the output dataframe to a csv file
  df_save.to_csv(csv_path + result_file_name, index = False) 
  print()



31/31 [==============================] - 2s 62ms/step
t5_large_ft.csv
unbalanced =  (array([0, 1, 2]), array([773, 211,   5]))
balanced =  (array([0, 1, 2]), array([115,   7, 867]))

31/31 [==============================] - 2s 64ms/step
bart_xsum_ft.csv
unbalanced =  (array([0, 1, 2]), array([798, 189,   2]))
balanced =  (array([0, 1, 2]), array([ 98,   6, 885]))

31/31 [==============================] - 2s 64ms/step
bart_cnn_ft.csv
unbalanced =  (array([0, 1, 2]), array([778, 208,   3]))
balanced =  (array([0, 1, 2]), array([107,   6, 876]))

31/31 [==============================] - 2s 64ms/step
bart_detox_zsl.csv
unbalanced =  (array([0, 1, 2]), array([781, 204,   4]))
balanced =  (array([0, 1, 2]), array([ 93,   9, 887]))



### Read CVS files, Display all cases for each CSV file

In [20]:
result_file_names = ['detox_bertweet_class_outoput_t5.csv','detox_bertweet_class_outoput_xsum.csv',
                     'detox_bertweet_class_outoput_cnn.csv','detox_bertweet_class_outoput_paradetox.csv']

In [21]:
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', None)

In [23]:
for i in range(len(result_file_names)):
  result_file_name = result_file_names[i]
  df_results = pd.read_csv(csv_path + result_file_name)
  df_results['class_type'] = df_results['pred_class'].map(str) + df_results['output_class'].map(str)
  print('\033[1m' + result_file_name + '\033[0m')
  print()
  print('class comparsion type counts')
  #print('class type counts = ', np.unique(df_results['class_type'], return_counts=True))
  df_group = df_results.groupby(list(df_results['class_type'])).size().reset_index()
  df_group.rename({0:'count'}, inplace=True, axis=1)
  display(df_group.T)
  print()

  # print first 10 examples in all cases 
  print('org_input = 0, output = 0')
  display(df_results.loc[df_results['class_type'] == '00', ['org_input', 'output']].head(10))
  print('org_input = 0, output = 1')
  display(df_results.loc[df_results['class_type'] == '01', ['org_input', 'output']].head(10))
  print('org_input = 0, output = 2')
  display(df_results.loc[df_results['class_type'] == '02', ['org_input', 'output']].head(10))
  print('org_input = 1, output = 0')
  display(df_results.loc[df_results['class_type'] == '10', ['org_input', 'output']].head(10))
  print('org_input = 1, output = 1')
  display(df_results.loc[df_results['class_type'] == '11', ['org_input', 'output']].head(10))
  print('org_input = 1, output = 2')
  display(df_results.loc[df_results['class_type'] == '12', ['org_input', 'output']].head(10))
  print('org_input = 2, output = 0')
  display(df_results.loc[df_results['class_type'] == '20', ['org_input', 'output']].head(10))
  print('org_input = 2, output = 1')
  display(df_results.loc[df_results['class_type'] == '21', ['org_input', 'output']].head(10))
  print('org_input = 2, output = 2')
  display(df_results.loc[df_results['class_type'] == '22', ['org_input', 'output']].head(10))
  print()
  print('==========================================================')
  print()


detox_bertweet_class_outoput_t5.csv

class comparsion type counts


,0,1,2,3,4,5,6,7
index,00,01,02,10,11,12,20,22
count,113,1,434,1,6,363,1,70



org_input = 0, output = 0


,org_input,output
2,can we apply that litmus test to muslims that kill in the name of their religion ? .,['can we apply that litmus test to muslims that kill in the name of their religion?']
14,do you not know what state secrets are or do you just pretend to be retarded ?,['Do you not know what state secrets are or do you just pretend to be retarded?']
19,then i 'm pretty sure isis is fucked .,"[""then i 'm pretty sure isis is messed up .""]"
20,where the fuck are the cops ?,['Where are the cops?']
29,what the fuck have u gt against ireland ? !,['What have you got against Ireland?']
44,barged my fucking way in and laid the smacketh down,['barged my way in and laid the smacketh down']
58,these clowns forget who they work for : the public .,['These people forget who they work for: the public.']
68,"the only sad thing , is that this moron weinstein got away with it for so long .","['the only sad thing , is that this weinstein got away with it for so long .']"
72,if ur bad bitch and a nigga ain 't saved u yet,"[""if ur bad and a nigga ain 't saved u yet""]"
83,lol that old man was funny as hell too i 'll beat both of ya ass like ya stole somthin,"[""lol that old man was funny too i 'll beat both of you like ya stole somthing""]"


org_input = 0, output = 1


...

,org_input,output
260,these four lane county commissioners are rotten to the core .,['These four lane county commissioners are corrupt']
510,well of course i flagged that load of garbage .,['Well of course I flagged that load of garbage.']
889,never take any responsibility and then post silly comments blaming liberals .,['Never take any responsibility and then post comments blaming liberals.']


org_input = 2, output = 1


,org_input,output


org_input = 2, output = 2


,org_input,output
6,this idea of just stacking things into legislation is stupid and ends up with government shutdowns .,['this idea of just stacking things into legislation is bad and ends up with government shutdowns .']
8,i seem to remember a certain dotard promising a cheaper and better health care program for all .,['i seem to remember a certain person promising a cheaper and better health care program for all .']
25,"damn , that is a lot of money",['That is a lot of money']
43,they kept the gifford crap going because they were trying to make sarah palin look bad .,['They kept the Gifford story because they were trying to make Sarah Palin look bad.']
46,you are going to die no matter if you do drugs or not .,['you are going to die no matter if you do drugs or not.']
48,but not as important as votegetting no matter how ridiculously hypocritical it seems,['But not as important as votegetting no matter how ridiculously hypocritical it seems']
55,tell her i said use da numba damn it ! !,['Tell her I said use the number']
66,the miami relatives and little havana are crazy enough to do this .,['The miami relatives and little havana are crazy enough to do this.']
81,the problem is that their management is so ridiculously incompetent .,['The problem is that their management is so incompetent.']
93,that way you wont have to keep blaming others for your foolishness .,['that way you wont have to keep blaming others for your wrong actions.']


In [ ]:
df_results = pd.read_csv(csv_path + result_file_name)

In [ ]:
df_results

,org_class,org_input,pred_class,output,output_class
0,1,"""Why would you wanna be the Green Ranger? He's...",0,Why would you wanna be the Green Ranger? He's ...,0
1,2,#HolySpirit God still share HIS #Secrets Amos ...,2,#HolySpirit God still share HIS #Secrets Amos ...,2
2,2,pancakes trash,2,pancakes are not good,2
3,2,"The KFAN mock draft continues, Cleveland is ""o...",2,"The KFAN mock draft continues, Cleveland is ""o...",2
4,1,I be telling Mcgirt music ain't enough.You got...,0,I be telling Mcgirt music ain't enough.You got...,2
...,...,...,...,...,...
2395,1,tell me why nick cannon bout to invite wiz to ...,1,tell me why nick cannon bout to invite wiz to ...,2
2396,1,"I just wanna touch da bitch, fuck da bitch, Li...",1,"I just wanna touch her, don't trust her.",2
2397,2,i dont think women should wear flesh tone colo...,2,i dont think women should wear flesh tone colo...,2
2398,1,You put nair on your pussy for the gram? Ok ht...,1,You put nair on you for the gram? Ok http://t....,2
